# Homework 3 -- Cloropleth map of grants in Swiss universities

In [1]:
# imports and all that
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os.path
import os, random
import time

### Data wrangling

In [2]:
grants = pd.read_csv("data/P3_GrantExport.csv", sep=';', index_col=0)
grants.size
grants.shape

(48464, 8)

We'll try to reduce the dataset by removing the entries that are not in CH and irrelevant columns. The foreign institutes don't have a valid entry in `University` field

These changes can be safely persisted in the file, as it will not influence the data we work with.

In [3]:
grants = grants[(grants.University.notnull()) & (grants.University != "Nicht zuteilbar - NA")]
grants = grants.drop(axis=1, errors='ignore', labels=['Project Title', 'Project Title English',
            'Responsible Applicant', 'Funding Instrument', 'Funding Instrument Hierarchy', 'Keywords'])

grants.dtypes
grants.shape

(48283, 8)

Some entries in the Approved Amount column are not numeric so they are being dropped

In [4]:
grants['Approved Amount'] = pd.to_numeric(grants['Approved Amount'], errors='coerce')
grants = grants[grants['Approved Amount'].notnull()]
grants.shape

(48283, 8)

In [5]:
# Run if you want to update the csv
#grants.to_csv("data/P3_GrantExport.csv", sep=';') # keep the same sep

We will group the results on university and combinate those with the NPO category that cannot be grouped on university.
This will allow us to llopu either the university or the institute to find a location.

In [6]:
universities = grants.groupby('University')
universities = universities.agg({'Approved Amount':'sum'}).reset_index()
universities = universities[~universities.University.str.startswith("NPO")]
universities.columns = ['Institution', 'Approved Amount']
universities.shape

(75, 2)

In [7]:
institutes = grants[grants.University.str.startswith("NPO")]
institutes = institutes.groupby('Institution').agg({'Approved Amount':'sum'}).reset_index()
institutes.shape

(450, 2)

Combine the two dataframes

In [8]:
frame = pd.concat([universities, institutes])

### Find location of institutes

Use Google API to find the location corresponding to the university. This will only run once and save the results of the requests into a file.

In [9]:
google_key="0" #todo insert google api key
data = 'data/requests.txt'
url = "https://maps.googleapis.com/maps/api/place/textsearch/xml?key="+google_key+"&query="

def get_city(row):
    r = requests.get(url+row.Institution)
    with open(data, "a") as myfile:
        myfile.write(r.text)
                
if not os.path.isfile(data): 
    print("Creating file %s" % data)
    with open(data, "a") as myfile:
        myfile.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?><results>")
    frame.apply(get_city, axis=1)
    with open(data, "a") as myfile:
        myfile.write("</results>")
else:
    print("File already exists %s" %data)

File already exists data/requests.txt


Load the results for the google requests and filter out the postalcodes of the Universities location. This will later be used to find the canton.

In [10]:
results = BeautifulSoup(open(data, encoding="latin_1"), "xml")
frame['postalcode'] = 'None'
i = 0
for response in results.find_all('PlaceSearchResponse'):
    result = response.find('result')
    if result != None:
        city = result.find('formatted_address').text
        if "Switzerland" in city:
            pc = str(city[city.index(',')+2:city.index(',')+6])
            if (pc.isdigit()):
                frame.set_value(i, 'postalcode', pc)
            else:
                frame.set_value(i, 'postalcode', city)
    i += 1
frame.head(10)

,Institution,Approved Amount,postalcode
0,AO Research Institute - AORI,3.435621e+06,None
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,None
2,Berner Fachhochschule - BFH,3.102870e+07,None
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,None
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,3012
5,EPF Lausanne - EPFL,1.175316e+09,3401
6,ETH Zürich - ETHZ,1.635597e+09,8280
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,4003
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,1950
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,None


For the universities that were found we will use the geonames api to find the canton based on the postalcode.

In [11]:
username = "juta" #todo add username
url = "http://api.geonames.org/postalCodeSearch?maxRows=1&username="+username+"&country=ch&postalcode="

def get_canton(row):
    if row.postalcode.isdigit():
        r = requests.get(url+row.postalcode)
        soup = BeautifulSoup(r.text, "xml")
        if soup.find('adminCode1'):
            return soup.find('adminCode1').text
        else:
            return 'None'
    else:
        return 'None'

frame['canton'] = frame.apply(get_canton, axis=1)
frame.head(10)

,Institution,Approved Amount,postalcode,canton
0,AO Research Institute - AORI,3.435621e+06,None,None
1,Allergie- und Asthmaforschung - SIAF,1.916996e+07,None,None
2,Berner Fachhochschule - BFH,3.102870e+07,None,None
3,Biotechnologie Institut Thurgau - BITG,2.492535e+06,None,None
4,Centre de rech. sur l'environnement alpin - CR...,1.567678e+06,3012,BE
5,EPF Lausanne - EPFL,1.175316e+09,3401,BE
6,ETH Zürich - ETHZ,1.635597e+09,8280,TG
7,Eidg. Anstalt für Wasserversorgung - EAWAG,7.461922e+07,4003,BS
8,"Eidg. Forschungsanstalt für Wald,Schnee,Land -...",4.836039e+07,1950,VS
9,Eidg. Hochschulinstitut für Berufsbildung - EHB,2.086572e+06,None,None


We save the information found to a file in order to manually add some missing locations

In [12]:
universities_data = "data/universities.csv"
if not os.path.isfile(universities_data):
    frame.to_csv(universities_data, sep=',', encoding='latin_1')
frame = pd.read_csv(universities_data, sep=',', encoding='latin_1')

In [13]:
df = frame.groupby(['canton'])
df = df.agg({'Approved Amount':'sum'}).reset_index()
df = df[df.canton != 'None']
df.to_csv("data/grants_per_canton.csv", sep=';')
df

,canton,Approved Amount
0,AG,1.197269e+06
1,BE,1.292178e+09
2,BL,4.924980e+05
3,BS,8.190666e+07
4,FR,2.285588e+06
5,GE,1.835588e+09
6,GR,7.308074e+07
7,JU,2.914904e+07
8,LU,1.851238e+09
9,NE,1.238273e+06
